# 202001549 장희권

In [23]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from konlpy.tag import Okt
from PIL import Image
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import nltk
from nltk.tokenize import word_tokenize
import math

In [3]:
driver = webdriver.Chrome(r"../../chromedriver.exe")
main_url = "https://movie.daum.net/main"
driver.get(main_url)

In [4]:
movie_name = '완벽한 타인'

time.sleep(1)

search_path = '//*[@id="mainContent"]/div/div[1]/form/fieldset/div/input'
driver.find_element("xpath", search_path).send_keys(movie_name)

time.sleep(1)

click_path = '//*[@id="mainContent"]/div/div[1]/div/div/div/ul/li[1]/a'
driver.find_element("xpath", click_path).click()

time.sleep(1)

gradeButton_path = '//*[@id="mainContent"]/div/div[2]/div[1]/ul/li[4]/a'
driver.find_element("xpath", gradeButton_path).click()

In [5]:
viewMore_path = '//*[@id="alex-area"]/div/div/div/div[3]/div[1]/button'

while True:
    time.sleep(1)
    try:
        driver.find_element("xpath", viewMore_path).click()
    except:
        break

In [11]:
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')

grade_list = []

cmt_lst = soup.find('ul', {"class":"list_comment"}).find_all('li')

sentence_class_name = "desc_txt"
ratings_class_name = "ratings"

avg_grade = float(soup.find('em', {'class':'num_rating'}).text[:-1])

for cmt in cmt_lst:
    try:
        rating = cmt.find('div', {'class':ratings_class_name}).text
        sentence = cmt.find('p', {'class':sentence_class_name}).text
        grade_list.append((int(rating), sentence))
    except:
        pass

In [12]:
bad_grade_list = [(rating, sentence) for rating, sentence in grade_list if rating < avg_grade]
good_grade_list = [(rating, sentence) for rating, sentence in grade_list if rating >= avg_grade]

In [13]:
okt = Okt()

In [14]:
grade_doc = '\n'.join(list(zip(*grade_list))[1])
grade_doc_noun = okt.nouns(grade_doc)

grade_count_noun = Counter(grade_doc_noun)
grade_count_noun.most_common(20)

[('영화', 823),
 ('생각', 235),
 ('연기', 216),
 ('배우', 164),
 ('사람', 135),
 ('것', 114),
 ('이서진', 112),
 ('비밀', 111),
 ('유해진', 109),
 ('이', 78),
 ('공감', 77),
 ('보고', 77),
 ('나', 75),
 ('재미', 75),
 ('더', 75),
 ('점', 73),
 ('수', 69),
 ('정말', 67),
 ('시간', 67),
 ('그냥', 65)]

In [16]:
stopword_list = {'보고', '정말', '그냥'}

for text in grade_count_noun.copy():
    if len(text) == 1:
        stopword_list.add(text)

for stopword in stopword_list:
    grade_count_noun.pop(stopword)
    
wc_img = WordCloud(background_color="white", max_words=2000, font_path=r'C:\Users\user\AppData\Local\Microsoft\Windows\Fonts\NanumSquare.ttf')
wc_img = wc_img.generate_from_frequencies(grade_count_noun)

plt.imshow(wc_img, interpolation="bilinear")
plt.axis('off')
plt.show()

KeyError: '정말'

In [ ]:
bad_grade_doc = '\n'.join(list(zip(*bad_grade_list))[1])
bad_grade_doc_noun = okt.nouns(bad_grade_doc)

bad_grade_count_noun = Counter(bad_grade_doc_noun)
bad_grade_count_noun.most_common(20)

In [ ]:
stopword_list = {'보고', '그냥'}

for text in bad_grade_count_noun.copy():
    if len(text) == 1:
        stopword_list.add(text)

for stopword in stopword_list:
    bad_grade_count_noun.pop(stopword)
    
wc_img = WordCloud(background_color="white", max_words=2000, font_path=r'C:\Users\user\AppData\Local\Microsoft\Windows\Fonts\NanumSquareR.ttf')
wc_img = wc_img.generate_from_frequencies(bad_grade_count_noun)

plt.imshow(wc_img, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
good_grade_doc = '\n'.join(list(zip(*good_grade_list))[1])
good_grade_doc_noun = okt.nouns(good_grade_doc)

good_grade_count_noun = Counter(good_grade_doc_noun)
good_grade_count_noun.most_common(20)

In [ ]:
stopword_list = {'보고', '정말', '간만'}

for text in good_grade_count_noun.copy():
    if len(text) == 1:
        stopword_list.add(text)

for stopword in stopword_list:
    good_grade_count_noun.pop(stopword)
    
wc_img = WordCloud(background_color="white", max_words=2000, font_path=r'C:\Users\user\AppData\Local\Microsoft\Windows\Fonts\NanumSquareR.ttf')
wc_img = wc_img.generate_from_frequencies(good_grade_count_noun)

plt.imshow(wc_img, interpolation="bilinear")
plt.axis('off')
plt.show()

In [223]:
class MyNaiveBayesClassifier:
    def __init__(self, k=0.5, use_morph=False):
        self.k = k
        self.word_prob_list = []
        self.word_ratio_list = []
        self.use_morph = use_morph
        
        if self.use_morph:
            self.okt = Okt()
            
    def load_data(self, grade_list):
        lst = [(text, 'pos') if grade >= avg_grade else (text, 'neg') for grade, text in grade_list]
        return zip(*lst)

    def tokenize(self, sentence):
        if self.use_morph:
            return ["{0}/{1}".format(word, tag) for word, tag in self.okt.pos(sentence, norm=True, stem=True)]
        else:
            return sentence.split()
    
    def count_words(self, docs, labels):
        count_dict = defaultdict(lambda:{'pos': 0, 'neg':0})
        for doc, label in zip(docs, labels):
            for word in self.tokenize(doc):
                count_dict[word][label] += 1
        
        print('num of words... {0}'.format(len(count_dict)))
        return count_dict
    
    def word_prob(self, count_dict, pos_class_num, neg_class_num, k):
        word_prob_list = []
        
        for word in count_dict.keys():
            pos_word_num = count_dict[word]['pos']
            neg_word_num = count_dict[word]['neg']
            
            pos_class_prob = (pos_word_num + k) / (pos_class_num + 2*k)
            neg_class_prob = (neg_word_num + k) / (neg_class_num + 2*k)
            
            word_prob_list.append((word, pos_class_prob, neg_class_prob))
        
        return word_prob_list
    
    def class_prob(self, word_prob_list, test_sentence, use_unseen=False):
        test_words = self.tokenize(test_sentence)
        
        sent_log_pos_class_prob, sent_log_neg_class_prob = 0.0, 0.0
        
        for word, word_pos_class_prob, word_neg_class_prob in word_prob_list:
            if word in test_words:
                sent_log_pos_class_prob += math.log(word_pos_class_prob) 
                sent_log_neg_class_prob += math.log(word_neg_class_prob)
            else:
                if use_unseen:
                    sent_log_pos_class_prob += math.log(1-word_pos_class_prob) 
                    sent_log_neg_class_prob += math.log(1-word_neg_class_prob)
            
        sent_pos_class_prob = math.exp(sent_log_pos_class_prob)
        sent_neg_class_prob = math.exp(sent_log_neg_class_prob)
        
        pos_class_prob = sent_pos_class_prob/(sent_pos_class_prob+sent_neg_class_prob)
        neg_class_prob = sent_neg_class_prob/(sent_pos_class_prob+sent_neg_class_prob)
        
        return pos_class_prob, neg_class_prob
    
    def word_ratio(self, word_prob_list):
        word_ratio_list = [(text, pos/pos, neg/pos) if pos < neg else (text, pos/neg, neg/neg) for text, pos, neg in self.word_prob_list]
        return list(sorted(word_ratio_list, key=lambda x: x[1]*x[2], reverse=True))
    
    def train(self, train_file_path):
        train_docs, train_labels = self.load_data(train_file_path)
        
        word_count_dict = self.count_words(train_docs, train_labels)
        
        pos_class_num = len([label for label in train_labels if label == 'pos'])
        neg_class_num = len([label for label in train_labels if label == 'neg'])
        
        self.word_prob_list = self.word_prob(word_count_dict, pos_class_num, neg_class_num, self.k)
        self.word_ratio_list = self.word_ratio(self.word_prob_list)
        
    def classify(self, test_sentence, use_unseen=False):
        pos_class_prob, neg_class_prob = self.class_prob(self.word_prob_list, test_sentence, use_unseen)
        
        if pos_class_prob > neg_class_prob:
            print('pos', pos_class_prob)
        else:
            print('neg', neg_class_prob)
    
    def show_most_informative_features(self, show_count=10, type="all"):
        count = 0
        for text, pos, neg in self.word_ratio_list:
            if count < show_count:
                if (type == "all") or  (type == "pos" and pos >= neg) or (type == 'neg' and neg > pos):
                    count += 1
                    print("{0} -> pos : neg = {1:.1f} : {2:.1f}".format(text, pos, neg))
            else:
                break

In [224]:
classifier = MyNaiveBayesClassifier(use_morph=True)

In [225]:
classifier.train(grade_list)

num of words... 4161


In [227]:
classifier.show_most_informative_features(type="all")

어색/Noun -> pos : neg = 1.0 : 35.0
의도/Noun -> pos : neg = 1.0 : 30.9
냐/Josa -> pos : neg = 1.0 : 30.9
수준/Noun -> pos : neg = 1.0 : 29.5
노잼/Noun -> pos : neg = 1.0 : 26.7
보헤미안/Noun -> pos : neg = 1.0 : 26.7
안좋다/Adjective -> pos : neg = 1.0 : 26.7
더럽다/Adjective -> pos : neg = 1.0 : 22.6
속이다/Verb -> pos : neg = 1.0 : 22.6
부인/Noun -> pos : neg = 1.0 : 22.6


In [228]:
classifier.classify('영화 재미있다', use_unseen=False)

pos 0.826862712175726
